# Shoulder Press 

### Importing Libraries

In [2]:
import cv2 as cv
import mediapipe as mp
import math

### Angle calcultion using three 2d Coordinates

In [3]:

def calculate_angle(x1, y1, x2, y2, x3, y3):
    delta_x1, delta_y1 = x1 - x2, y1 - y2
    delta_x2, delta_y2 = x3 - x2, y3 - y2
    angle_rad = math.atan2(delta_y2, delta_x2) - math.atan2(delta_y1, delta_x1)
    angle_deg = math.degrees(angle_rad)
    angle_deg = (angle_deg + 360) % 360
    return angle_deg

In [8]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


### Fucntion to detect the state of Shoulder Press (up or down)

In [9]:
def detect_shoulder_press_state(frame, landmarks, initCheck, state, rep_count):

    initial_position = (20, 150)
    line_height = 30
    y = initial_position[1]
    
    # Necessary Landmarks Detection  
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Calculating Angles
    left_elbow_angle = calculate_angle(left_shoulder.x, left_shoulder.y,
                                    left_elbow.x, left_elbow.y,
                                    left_wrist.x, left_wrist.y)
        
    left_shoulder_angle = calculate_angle(left_elbow.x, left_elbow.y, 
                                    left_shoulder.x, left_shoulder.y,
                                    left_hip.x, left_hip.y)

    right_elbow_angle = calculate_angle(right_wrist.x, right_wrist.y,
                                    right_elbow.x, right_elbow.y,
                                    right_shoulder.x, right_shoulder.y)

    
    right_shoulder_angle = calculate_angle(right_hip.x, right_hip.y,
                                     right_shoulder.x, right_shoulder.y,
                                     right_elbow.x, right_elbow.y)
    
    if(initCheck == 1 and ((55 < left_elbow_angle > 75) and (55 < right_elbow_angle < 75) and (90 < left_shoulder_angle > 110 ) and (90 < right_shoulder_angle < 110) and ((left_wrist.y + 30) > nose.y and (right_wrist.y + 30) > nose.y))):
        initCheck = 0
        state == "startway"
        cv.putText(frame, f"Great you're in Initial position", (10, 330), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 0, cv.LINE_AA)
    else:
        text = "Attain initial stage: Bend your elbows \npointing directly under your wrists."
        lines = text.split('\n')

 
        cv.rectangle(frame, (15, 105), (220, 150), (255, 255, 255), cv.FILLED)
        # for line in lines:
        #     cv.putText(frame, line, (initial_position[0], y), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv.LINE_AA)
        #     y += line_height

    if initCheck == 0:

        if (state == "startway") and (125 < (left_elbow_angle and right_elbow_angle) < 145) and (150 < (left_shoulder_angle and right_shoulder_angle) < 170) and ((left_elbow.y+40 > nose.y) and (right_elbow.y+40 > nose.y)):

            state = "endway"
            rep_count += 1
            # cv.putText(frame, f"Rep {rep_count} ", (100, 330), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)

        elif state == "endway" and (55 < (left_elbow_angle and right_elbow_angle) < 75) and (90 <(left_shoulder_angle and right_elbow_angle) < 110)((left_wrist.y+40 > nose.y) and (right_wrist.y+40 > nose.y)):
            state = "startway"
    
    return frame, initCheck, state, rep_count, 

### Pose estimator

In [10]:

video_path = r"demos/ShoulderPress/shoulderPressV"
cap = cv.VideoCapture(video_path)

buffer_size = 5
initCheck = 1
frame_buffer = []

frame_skip = 4 
count = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    count += 1 
    if count % frame_skip != 1:
        continue

    frame = cv.resize(frame, (640, 480))

    # image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    results = pose.process(frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        if not frame_buffer:
            state = "startway"
            rep_count = 0
        else:
            state, rep_count = frame_buffer[-1] 

        frame, initCheck, state, rep_count = detect_shoulder_press_state(frame, landmarks, initCheck, state, rep_count)


        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=4, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2) 
                                 )
        cv.putText(frame, f"CORRECT POSTURE", (28, 135), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 100, 0), 0, cv.LINE_AA)
        
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        cv.imshow('Exercise Detection', frame)
        cv.imwrite('Shoulder Press Correct.jpg', frame)

        frame_buffer.append((state, rep_count))
        if len(frame_buffer) > buffer_size:
            frame_buffer.pop(0)

        if cv.waitKey(0) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()